In [1]:
# Import packages
import arcpy
import fileinput
import numpy as np
import os

# Set overwrite option
arcpy.env.overwriteOutput = True

# Set arcpy working environment
arcpy.env.workspace = "K:/VegetationEcology/BristolBay_Vegetation/Project_GIS/BristolBay_Vegetation.gdb"

In [2]:
# Define a wrapper function for arcpy geoprocessing tasks
def arcpy_geoprocessing(geoprocessing_function, check_output = True, check_input = True, **kwargs):
    """
    Description: wraps arcpy geoprocessing and data access functions for file checks, message reporting, and errors.
    Inputs: geoprocessing function -- any arcpy geoprocessing or data access processing steps defined as a function that receive ** kwargs arguments.
            check_output -- boolean input to control if the function should check if the output already exists prior to executing geoprocessing function
            check_input -- boolean input to control if the function should check if the inputs already exist prior to executing geoprocessing function
            **kwargs -- key word arguments that are used in the wrapper and passed to the geoprocessing function
                'input_array' -- if check_input == True, then the input datasets must be passed as an array
                'output' -- if check_output == True, then the output dataset must be defined
    Returned Value: Function returns messages, warnings, and errors from geoprocessing functions.
    Preconditions: geoprocessing function and kwargs must be defined, this function does not conduct any geoprocessing on its own
    """
    try:
        # If check_output is True, then check if output exists and warn of overwrite if it does
        if check_output == True and arcpy.Exists(kwargs['output']):
            print(kwargs['output'] + " already exists and will be overwritten")
        # If check_input is True, then check if inputs exist and quit if any do not
        if check_input == True:
            for input_data in kwargs['input_array']:
                if arcpy.Exists(input_data) != True:
                    print(input_data + " does not exist. Check that environment workspace is correct.")
                    quit()
        # Execute geoprocessing function if all input data exists
        out_process = geoprocessing_function(**kwargs)
        # Provide results report
        try:
            msg_count = out_process.messageCount
            print(out_process.getMessage(msg_count - 1))
        except:
            print(out_process)
    # Provide arcpy errors for execution error
    except arcpy.ExecuteError as err:
        print(arcpy.GetMessages())

In [3]:
# Define function to create polygon feature class
def create_polygon(**kwargs):
    """
    Description: creates a polygon feature class with vertices read from an array
    Inputs: 'point_array' -- an array of x, y coordinates
            'output' -- the output should be a feature class
            'output_path' -- the output path is the directory in which the new feature class is created
    """
    # Create a new polygon feature class
    arcpy.CreateFeatureclass_management(kwargs['output_path'],
                                        os.path.split(kwargs['output'])[1],
                                        'POLYGON')
    # Define spatial reference and add projection to new feature class
    spatial_reference = arcpy.SpatialReference(kwargs['spatial_reference'])
    output_file = os.path.join(kwargs['output_path'], kwargs['output'])
    arcpy.DefineProjection_management(output_file, spatial_reference)
    # Initiate an insert cursor and insert an array of vertices for a polygon
    with arcpy.da.InsertCursor(output_file, ["SHAPE@"]) as cursor:
        array = arcpy.Array()
        for point in kwargs['point_array']:
            X = point[0]
            Y = point[1]
            array.add(arcpy.Point(X, Y))
        cursor.insertRow([arcpy.Polygon(array)])
    # Return success message to the user
    print("------------")
    out_process = "Successfully created polygon feature"
    return out_process

In [4]:
# Define input array
point_array = [[-160.00218, 61.52927],
               [-152.79699, 61.07407],
               [-153.07435, 58.82993],
               [-157.69984, 56.33955],
               [-159.49354, 56.97233],
               [-163.09044, 58.59629],
               [-160.84181, 61.57616]]

# Define output feature class
output_path = arcpy.env.workspace
model_area = "modelArea"

# Create key word arguments
create_polygon_kwargs = {'point_array' : point_array,
                         'output' : model_area,
                         'output_path' : output_path,
                         'spatial_reference' : 4326}

# Process the create polygon function with the point array
arcpy_geoprocessing(create_polygon, check_input = False, **create_polygon_kwargs)

------------
Successfully created polygon feature
